In [1]:
# step one
import sys, os
sys.path.append(os.path.abspath(".."))
from src.modeling import train_and_track_models

In [2]:
#  Step 2: Define Paths and Parameters
data_path = "../data/processed/features_with_target.csv"   
target_col = "is_high_risk"       # Proxy target created in Task 4
id_col = "CustomerId"
model_dir = "../models"
report_dir = "../reports"
mlflow_experiment_name = "credit-risk-models"
random_state = 42

In [3]:
#  Step 3: Train Models & Track with MLflow
best_model_name, best_model, best_f1_score = train_and_track_models(
    data_path=data_path,
    target_col=target_col,
    id_col=id_col,
    model_dir=model_dir,
    report_dir=report_dir,
    mlflow_experiment_name=mlflow_experiment_name,
    random_state=random_state
)


Data loaded: 3742 rows, 14 cols
Missing values per column before processing:
Amount_sum                 0
Amount_mean                0
Amount_std               712
Amount_count               0
txn_year_nunique           0
txn_month_nunique          0
txn_dayofweek_nunique      0
txn_hour_nunique           0
ProviderId                 0
ProductCategory            0
ChannelId                  0
PricingStrategy            0
dtype: int64
Missing values detected and will be handled via imputation in pipeline.
Numerical columns: ['Amount_sum', 'Amount_mean', 'Amount_std', 'Amount_count', 'txn_year_nunique', 'txn_month_nunique', 'txn_dayofweek_nunique', 'txn_hour_nunique', 'PricingStrategy']
Categorical columns: ['ProviderId', 'ProductCategory', 'ChannelId']
Train size: 2993, Test size: 749

Training model: logistic_regression
Best Params: {'clf__C': 0.01}
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.54      0.67       463
     

/home/nurye/Desktop/10_Academy/week_5/Credit-Risk-Probability-Model-for-Alternative-Data/CS-venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/06/27 10:19:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best Params: {'clf__learning_rate': 0.1, 'clf__max_depth': 3, 'clf__n_estimators': 200}
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.63      0.74       463
           1       0.59      0.88      0.71       286

    accuracy                           0.72       749
   macro avg       0.75      0.76      0.72       749
weighted avg       0.78      0.72      0.73       749

Confusion Matrix:
 [[290 173]
 [ 33 253]]
Accuracy: 0.7250
Precision: 0.5939
Recall: 0.8846
F1 Score: 0.7107
ROC AUC: 0.8211

Best model: random_forest with F1 score: 0.7124


Registered model 'random_forest_final' already exists. Creating a new version of this model...
Created version '4' of model 'random_forest_final'.


In [ ]:
#  Step 4: Summary
print(f" Best Model: {best_model_name}")
print(f" Best F1 Score: {best_f1_score:.4f}")

 Best Model: random_forest
 Best F1 Score: 0.7124
